In [ ]:
from bs4 import BeautifulSoup
import requests
import pickle
import urllib.request
import os

# This variable contains the links which car types you would like to .... ?
# [ Fabia, Octavia, SuperB ]
car_types = [
            "https://www.hasznaltauto.hu/talalatilista/auto/0P5WA5O3L3YQM583L5U61285URLHHYM3OJDPO6T3URLQJGU28LLURQ95RWEEETSOF6O9IEW9QEWEQQG1G7RC69IOIWHK3F0SWYROQ9MROPR7S11H3U8T9U3PG4152L08PEWH9T35ROF6TY6K671Y8YFOWEOMTG9GL3C2REKG/page",
            #"https://www.hasznaltauto.hu/talalatilista/auto/KL85WLTWU1YTY6Y6IEQ6K36E49MD9L6TALYIGPDKP4K85FY3E2COO9YOSF0TK3ZT6CJHE9GUJQ3ZU66HUEH1YH35QUYFP3ODQ3J9SUW1WEGPPGHLZ726OJR0UI7W84LTT44Q7RDMEIQYQTFKGI8CE7CUEMCUCMIO5JLSY/page",
            #"https://www.hasznaltauto.hu/talalatilista/auto/YHUQ5HD5JESDSGSG237GYWG3PR0KRHGDFHS2Z6KY6PYUQCSW3O944RS4LCIDYW8DG9AM3RZJA7W8JGGMJ3MESMWQ7JSC6W4K7WARLEKS3Z365ZIEUARUG65W10L3PGKMGMAYMC0Y3OT7RLYA2G63QM799MT9ES235L6F0S/page"
            ]

class car_data:
    model = ""
    first_registration_from = -1    
    """ ... """
    # def set_variables(self, ...):

# This class is responsible for downloading our database.
class data_miner:
    
    # ------------------------------------------------ Variables ------------------------------------------------ #    
    # Variables for reaching the website
    website = ""
    cookies = dict( BCPermissionLevel = 'PERSONAL' ) # With this you will not get cookie warning page
    headers = { "User-Agent" : "Mozilla/5.0" }
    
    # Variables for web
    page_count = -1
    car_links = []
    car_image_links = []
    
    # ------------------------------------------------ Functions ------------------------------------------------ #
    # This function get the number of car pages
    def get_page_count(self):
        self.page_count = -1
        page = requests.get( self.website + "1", headers = self.headers, cookies = self.cookies )
        soup = BeautifulSoup( page.content, 'lxml' )
        for link in soup.find_all("a", {"title" : "Utolsó oldal"}):
            self.page_count = int(link.getText())
        print("%s \n<- links's page count is: %s" % (self.website, self.page_count))
        # assert self.page_count > -1      # If the page does not contain "Utolsó oldal" element, the page_count's value equals -1
        self.get_car_links()
        
    # This function get all of the car's link on a page
    def get_car_links(self):
        for page_number in range(1,self.page_count + 1):
            page = requests.get( self.website + str(page_number), headers = self.headers, cookies = self.cookies )
            soup = BeautifulSoup( page.content, 'lxml' )
            
            car_count_per_page = 1 # Just for printing the state
            for link in soup.select(".talalati_lista .talalati_lista_head h2 a"):
                car_link = link.get("href")
                print("Page %d's %s. car: %s" % (page_number, car_count_per_page, car_link))
                self.car_links.append(car_link)
                self.get_car_information(car_link)
                car_count_per_page += 1
                
        print(self.car_links)
        # self.save_car_links()
        
    # This function save the image and car data
    def get_car_information(self, _car_link):        
        page = requests.get( _car_link, headers = self.headers, cookies = self.cookies )
        soup = BeautifulSoup( page.content, 'lxml' )
        
        # Get the image links
        # EZEK CSAK A PICI KÉPEK, AMIKET MEGJELENÍT ALUL VISZONT!!!!!! AHOGY NÉZTEM NINCS RAJTUK RÁÉGETETT FELIRAT
        # A MÉRETE IS 118x88 -AS, AMI SZERINTEM NEM IS ROSSZ KEZDÉSNEK
        image_counter = 1 # Just for printing the state
        for link in soup.select(".img_page a img"): 
            image_link = link.get("src")
            if image_link not in self.car_image_links:
                self.car_image_links.append(image_link)
                
                # Save image. The folder name equals the car's unique 8digit key
                image_name = _car_link[len(_car_link) - 8: ]
                folder_name = "images/" + image_name
                
                if not os.path.exists(folder_name):
                    os.makedirs(folder_name)
                    
                urllib.request.urlretrieve(image_link, folder_name + "/" + image_name + "_" + str(image_counter) + ".jpg")
                print("%d. image ( %s ) is saved!" % (image_counter, image_link) )
                image_counter += 1
                
        # Get car's data
        # Ide kéne megírni, a többi adat leszedéséhez szükséges kódot (évszám, stb..)
        # Szerinted csináljunk külön adatstruktúrát egy autónak? - car_data -
        # Szerintem úgy könnyebb lenne kezelni az adatokat (mentés - beolvasás)
        current_car = car_data()
        # TODO get car's data
        # save_car_data(current_car)
        
    # Saves a "car_data" object
    def save_car_data(self, _obj, _file_name):
        with open(_file_name + '.pkl', 'wb') as output:
            pickle.dump(_obj, output, pickle.HIGHEST_PROTOCOL)
            
    # Loads a "car_data" object
    def load_car_data(self, _file_name):
        with open(_file_name + '.pkl', 'rb') as input:
            return pickle.load(input) # This is a car_data type
            
    # After the algorithm has iterated through all of the cars this function saves the links to file
    # Ez valószínűleg felesleges, mert ha leszednek egy kocsit, úgy sem fog élni a linkje :D
    # És ha leszedünk valamit újra, akkor úgyis újból végigiterálsz az egészen :D
    def save_car_links(self):
        string = ""
        for car in self.car_links:
            string += a + '\n'
            
        text_file = open("asd.txt", "w")
        text_file.write(string)
        text_file.close()
        
def main():
    dm = data_miner()
    for car_type in car_types:
        dm.website = car_type
        dm.get_page_count()
    
if __name__ == "__main__":
    main()

https://www.hasznaltauto.hu/talalatilista/auto/0P5WA5O3L3YQM583L5U61285URLHHYM3OJDPO6T3URLQJGU28LLURQ95RWEEETSOF6O9IEW9QEWEQQG1G7RC69IOIWHK3F0SWYROQ9MROPR7S11H3U8T9U3PG4152L08PEWH9T35ROF6TY6K671Y8YFOWEOMTG9GL3C2REKG/page 
<- links's page count is: 81
Page 1's 1. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_1.2._mo-i._szervo-12024089
1. image ( https://hasznaltauto.medija.hu/10665/12024089_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/10665/12024089_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/10665/12024089_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/10665/12024089_4t.jpg ) is saved!
5. image ( https://hasznaltauto.medija.hu/10665/12024089_5t.jpg ) is saved!
6. image ( https://hasznaltauto.medija.hu/10665/12024089_6t.jpg ) is saved!
7. image ( https://hasznaltauto.medija.hu/10665/12024089_7t.jpg ) is saved!
8. image ( https://hasznaltauto.medija.hu/10665/12024089_8t.jpg ) is saved!
9. image ( https://hasznaltauto.medija.h

1. image ( https://hasznaltauto.medija.hu/1707062/12028593_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/1707062/12028593_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/1707062/12028593_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/1707062/12028593_4t.jpg ) is saved!
5. image ( https://hasznaltauto.medija.hu/1707062/12028593_5t.jpg ) is saved!
6. image ( https://hasznaltauto.medija.hu/1707062/12028593_6t.jpg ) is saved!
7. image ( https://hasznaltauto.medija.hu/1707062/12028593_7t.jpg ) is saved!
8. image ( https://hasznaltauto.medija.hu/1707062/12028593_8t.jpg ) is saved!
9. image ( https://hasznaltauto.medija.hu/1707062/12028593_9t.jpg ) is saved!
10. image ( https://hasznaltauto.medija.hu/1707062/12028593_10t.jpg ) is saved!
11. image ( https://hasznaltauto.medija.hu/1707062/12028593_11t.jpg ) is saved!
12. image ( https://hasznaltauto.medija.hu/1707062/12028593_12t.jpg ) is saved!
13. image ( https://hasznaltauto.medija.hu/static/images/v

9. image ( https://hasznaltauto.medija.hu/10665/12024461_9t.jpg ) is saved!
10. image ( https://hasznaltauto.medija.hu/10665/12024461_10t.jpg ) is saved!
11. image ( https://hasznaltauto.medija.hu/10665/12024461_11t.jpg ) is saved!
12. image ( https://hasznaltauto.medija.hu/10665/12024461_12t.jpg ) is saved!
Page 2's 8. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_1.2._mo-i-12024606
1. image ( https://hasznaltauto.medija.hu/10665/12024606_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/10665/12024606_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/10665/12024606_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/10665/12024606_4t.jpg ) is saved!
5. image ( https://hasznaltauto.medija.hu/10665/12024606_5t.jpg ) is saved!
6. image ( https://hasznaltauto.medija.hu/10665/12024606_6t.jpg ) is saved!
7. image ( https://hasznaltauto.medija.hu/10665/12024606_7t.jpg ) is saved!
8. image ( https://hasznaltauto.medija.hu/10665/12024606_8t.jpg )

5. image ( https://hasznaltauto.medija.hu/10333/11848145_5t.jpg ) is saved!
6. image ( https://hasznaltauto.medija.hu/10333/11848145_6t.jpg ) is saved!
7. image ( https://hasznaltauto.medija.hu/10333/11848145_7t.jpg ) is saved!
8. image ( https://hasznaltauto.medija.hu/10333/11848145_8t.jpg ) is saved!
9. image ( https://hasznaltauto.medija.hu/10333/11848145_9t.jpg ) is saved!
10. image ( https://hasznaltauto.medija.hu/10333/11848145_10t.jpg ) is saved!
11. image ( https://hasznaltauto.medija.hu/10333/11848145_11t.jpg ) is saved!
12. image ( https://hasznaltauto.medija.hu/10333/11848145_12t.jpg ) is saved!
Page 3's 7. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_1.2_tsi._ambition._afas._mo-i-12023965
1. image ( https://hasznaltauto.medija.hu/10665/12023965_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/10665/12023965_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/10665/12023965_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/106

1. image ( https://hasznaltauto.medija.hu/2008146/11830716_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/2008146/11830716_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/2008146/11830716_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/2008146/11830716_4t.jpg ) is saved!
5. image ( https://hasznaltauto.medija.hu/2008146/11830716_5t.jpg ) is saved!
6. image ( https://hasznaltauto.medija.hu/2008146/11830716_6t.jpg ) is saved!
Page 4's 8. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_1.9_sdi_comfort-12029081
Page 4's 9. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_sedan_1.4_classic-12030071
1. image ( https://hasznaltauto.medija.hu/13294/12030071_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/13294/12030071_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/13294/12030071_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/13294/12030071_4t.jpg ) is saved!
5. image ( https://hasznalta

...

9. image ( https://hasznaltauto.medija.hu/364/11989242_9t.jpg ) is saved!
10. image ( https://hasznaltauto.medija.hu/364/11989242_10t.jpg ) is saved!
11. image ( https://hasznaltauto.medija.hu/364/11989242_11t.jpg ) is saved!
12. image ( https://hasznaltauto.medija.hu/364/11989242_12t.jpg ) is saved!
Page 73's 9. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_1.0_cool-11863009
1. image ( https://hasznaltauto.medija.hu/7010/11863009_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/7010/11863009_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/7010/11863009_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/7010/11863009_4t.jpg ) is saved!
Page 73's 10. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_1.0_cool-11738030
1. image ( https://hasznaltauto.medija.hu/7784/11738030_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/7784/11738030_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/7784/11738030_3t.jp

3. image ( https://hasznaltauto.medija.hu/2601/11832094_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/2601/11832094_4t.jpg ) is saved!
5. image ( https://hasznaltauto.medija.hu/2601/11832094_5t.jpg ) is saved!
6. image ( https://hasznaltauto.medija.hu/2601/11832094_6t.jpg ) is saved!
Page 75's 5. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_cool_1.0_75le_rendelheto-10231553
1. image ( https://hasznaltauto.medija.hu/2452/10231553_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/2452/10231553_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/2452/10231553_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/2452/10231553_4t.jpg ) is saved!
5. image ( https://hasznaltauto.medija.hu/2452/10231553_5t.jpg ) is saved!
6. image ( https://hasznaltauto.medija.hu/2452/10231553_6t.jpg ) is saved!
7. image ( https://hasznaltauto.medija.hu/2452/10231553_7t.jpg ) is saved!
8. image ( https://hasznaltauto.medija.hu/2452/10231553_8t.jpg ) i

11. image ( https://hasznaltauto.medija.hu/377/11800847_11t.jpg ) is saved!
12. image ( https://hasznaltauto.medija.hu/377/11800847_12t.jpg ) is saved!
Page 76's 7. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_1.2_tsi_ambition_afa-s_szamla-11792369
1. image ( https://hasznaltauto.medija.hu/1138/11792369_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/1138/11792369_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/1138/11792369_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/1138/11792369_4t.jpg ) is saved!
5. image ( https://hasznaltauto.medija.hu/1138/11792369_5t.jpg ) is saved!
6. image ( https://hasznaltauto.medija.hu/1138/11792369_6t.jpg ) is saved!
7. image ( https://hasznaltauto.medija.hu/1138/11792369_7t.jpg ) is saved!
8. image ( https://hasznaltauto.medija.hu/1138/11792369_8t.jpg ) is saved!
9. image ( https://hasznaltauto.medija.hu/1138/11792369_9t.jpg ) is saved!
10. image ( https://hasznaltauto.medija.hu/1138/11792369_10

1. image ( https://hasznaltauto.medija.hu/2601/11832141_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/2601/11832141_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/2601/11832141_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/2601/11832141_4t.jpg ) is saved!
Page 77's 9. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_combi_1.0_clever-11995815
1. image ( https://hasznaltauto.medija.hu/7784/11995815_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/7784/11995815_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/7784/11995815_3t.jpg ) is saved!
4. image ( https://hasznaltauto.medija.hu/7784/11995815_4t.jpg ) is saved!
5. image ( https://hasznaltauto.medija.hu/7784/11995815_5t.jpg ) is saved!
6. image ( https://hasznaltauto.medija.hu/7784/11995815_6t.jpg ) is saved!
7. image ( https://hasznaltauto.medija.hu/7784/11995815_7t.jpg ) is saved!
8. image ( https://hasznaltauto.medija.hu/7784/11995815_8t.jpg ) is saved!

5. image ( https://hasznaltauto.medija.hu/2411/11922043_5t.jpg ) is saved!
6. image ( https://hasznaltauto.medija.hu/2411/11922043_6t.jpg ) is saved!
7. image ( https://hasznaltauto.medija.hu/2411/11922043_7t.jpg ) is saved!
8. image ( https://hasznaltauto.medija.hu/2411/11922043_8t.jpg ) is saved!
9. image ( https://hasznaltauto.medija.hu/2411/11922043_9t.jpg ) is saved!
10. image ( https://hasznaltauto.medija.hu/2411/11922043_10t.jpg ) is saved!
11. image ( https://hasznaltauto.medija.hu/2411/11922043_11t.jpg ) is saved!
Page 79's 3. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_combi_1.0_clever-11523949
Page 79's 4. car: https://www.hasznaltauto.hu/auto/skoda/fabia/skoda_fabia_combi_1.2_clever_dsg_valto-11870346
1. image ( https://hasznaltauto.medija.hu/2778508/11870346_1t.jpg ) is saved!
2. image ( https://hasznaltauto.medija.hu/2778508/11870346_2t.jpg ) is saved!
3. image ( https://hasznaltauto.medija.hu/2778508/11870346_3t.jpg ) is saved!
4. image ( https://haszna

In [ ]:

2. image ( https://hasznaltauto.medija.hu/1119/11877293_2t.jpg ) is saved!
